# [1.1] Get the list of movies

Let's start downloading the html files with inside the urls of the documents we need!

In [57]:
from bs4 import BeautifulSoup
import requests
import json

In [58]:
soup = BeautifulSoup(open('C:/Users/leona/Desktop/ADMHMK-3/movies2.html'), "html.parser")
soup.head()
lst_a = soup.select('a')
urls = []
for i in lst_a:
    urls.append(i.get('href'))
urls[0]

'https://en.wikipedia.org/wiki/10_to_Midnight'

In [59]:
soup = BeautifulSoup(open('C:/Users/leona/Desktop/ADMHMK-3/movies1.html'), "html.parser")
soup.head()
lst_a = soup.select('a')
lst_a
for i in lst_a:
    urls.append(i.get('href'))
urls[10000]   

'https://en.wikipedia.org/wiki/Love_by_the_Light_of_the_Moon'

In [60]:
soup = BeautifulSoup(open('C:/Users/leona/Desktop/ADMHMK-3/movies3.html'), "html.parser")
soup.head()
lst_a = soup.select('a')
lst_a
for i in lst_a:
    urls.append(i.get('href'))
urls[20000] 

'https://en.wikipedia.org/wiki/Z.P.G.'

In [61]:
urls[29999]

'https://en.wikipedia.org/wiki/Whistle_(2003_film)'

Now that we have all 30.000 urls we need to save them in dictionary that we will access later when we want to print the urls in the output of the search engines.

In [70]:
dicturls = {}
for i in range(len(urls)):
    dicturls[i] = urls[i]
with open('dicturls.json', 'w') as fp:
    json.dump(dicturls, fp)
with open(r"C:\Users\leona\Desktop\ADMHMK-3\dicturls.json", 'r') as file:
    data = file.read()
dicturls = json.loads(data) 

In [72]:
dicturls[str(29999)]

'https://en.wikipedia.org/wiki/Whistle_(2003_film)'

# [1.2] Crawl Wikipedia

Now, avoiding any error,(there will not be, and knowing this I already saved in order the urls), we download on our pc the html files we need for our database.

In [ ]:
from urllib.error import URLError, HTTPError, ContentTooShortError
import time
def getwikipageshtml(urls):
    k=0
    for i in range(len(urls)):
        try:
            ur_l = requests.get(urls[i])
            soup = BeautifulSoup(ur_l.content, 'html.parser')
            soup = soup.prettify("utf-8")   
            stringa = 'Articles/article-'+str(k)+'.html'
            k = k+1
            Html_file= open(stringa, "wb")
            Html_file.write(soup)
            Html_file.close()
        except(URLError,HTTPError, ContentTooShortError)  as e:
            html = None
        #time.sleep(0.001) #Actually for this task we don't need to stop anytime
    return
getwikipageshtml(urls)

# [1.3] Parse downloaded pages

Here we access every html document and take out the informations we need to build our tsv files with info title, intro, plot, film_name, director, producer, writer, starring, music, release date, runtime, country, language, budget.

In [8]:
import csv
import io
for i in range(30000):
    filename = "Articles/article-"+str(i)+".html"
    with open(filename, encoding="utf-8") as f:
        data = f.read()
        soup = BeautifulSoup(data, 'html.parser')
        #Now That I opened the file I have to look for the section asked
        #first I take the title and clear him of spaces and the word -Wikipedia
        titlepage = soup.title.string
        titleonly = titlepage.split("- Wikipedia")
        titlepage = titleonly[0].strip()
        #now I create empty string as intro and plot
        intro = ''
        plot = ''
        #Now i searcvh the first paragraph that usually or is empty or is the intro
        start = soup.find('p')
        intro = start
        intro1 = start.text
        B = ''
        #in B I put m,y limit for the paragraphs in the intro, because after this h2 there will always be the plot
        B = intro.find_next_sibling('h2')
        if(B!=None and B.find_next_sibling('p')):
            C = B.find_next_sibling('p')
            while(C != intro.find_next_sibling('p')): 
                intro1 = intro1 + intro.find_next_sibling('p').text
                intro = intro.find_next_sibling('p')
            plot = ''    
            #then i do the same with the plot, so I start at B and end in the next h2
            plot = B
            plot1 = ''
            compare = ''
            if(B.find_next_sibling('h2')):
                compare = B.find_next_sibling('h2')
                compareto = compare.find_next_sibling('p')
                while(compareto != plot.find_next_sibling('p')):
                    plot1 = plot1 + plot.find_next_sibling('p').text
                    #print(plot1)
                    plot = plot.find_next_sibling('p')
                    #if plot or intro are empty I put NA
        if(intro1 == ''):
            intro1 = "NA"
        if plot1 == '':
            plot1 = "NA"
        #Now I start working on the other features
        intro = intro1
        plot = plot1
        film_name = 'NA'
        director = "NA"
        producer = "NA"
        writer = "NA"
        starring = "NA"
        music = "NA"
        release_date = "NA"
        runtime = "NA"
        country = "NA"
        language = "NA"
        budget = "NA"
#'film_name', 'director', 'producer','writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget'
        for link in soup.find_all('tr'):
            if soup.find('th',{'class': ['summary']})!= None:
                    film_name = soup.find('th',{'class': ['summary']} ).text.strip()
            if link.th:
#I just check in the th and if I find the class I need I take the relative td. Some of them are inaccessible so the if link.td
                if(link.th.text.strip() == 'Directed by'):
                     director = link.td.text.strip()
                elif(link.th.text.strip() == 'Produced by'):
                      producer = link.td.text.strip()
                elif(link.th.text.strip() == 'Written by'):
                    writer = link.td.text.strip()
                elif(link.th.text.strip() == 'Starring'):
                    starring = link.td.text.strip()
                elif(link.th.text.strip() == 'Music by'):
                     music = link.td.text.strip()               
                elif(link.th.text.strip() == 'Release date'):
                    if link.td:
                        release_date = link.td.text.strip()
                elif(link.th.text.strip() == 'Running time'):
                    runtime = link.td.text.strip()
                elif(link.th.text.strip() == 'Country'):
                    if link.td:
                        country = link.td.text.strip()
                elif(link.th.text.strip() == 'Language'):
                    if link.td:
                        language = link.td.text.strip()
                elif(link.th.text.strip() == 'Budget'):
                    budget = link.td.text.strip()
#now I open the tsv files and create one for every film.        
        tsvname = 'Tsvfiles/'+'film'+str(i)+'.tsv'
        with io.open(tsvname, "w", encoding="utf-8") as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(['title','intro', 'plot', 'film_name', 'director', 'producer','writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget'])
            tsv_writer.writerow([titlepage, intro, plot, film_name, director, producer, writer, starring, music, release_date, runtime, 
                 country, language, budget])

In [9]:
titlepage

'Whistle (2003 film)'

In [17]:
intro

'\n\n\n\n         What Became of Jack and Jill?\n        \n\n       is a 1972 British\n       \n        horror film\n       \n       directed by\n       \n        Bill Bain\n       \n       and starring\n       \n        Mona Washbourne\n       \n       ,\n       \n        Paul Nicholas\n       \n       , and\n       \n        Vanessa Howard\n       \n       .\n       \n\n         [1]\n        \n\n       It was part of an abandoned attempt by\n       \n        Amicus Pictures\n       \n       to compete with\n       \n        Hammer Studios\n       \n       by breaking into the\n       \n        grindhouse\n       \n       market. Studio executives were ultimately too disturbed by the final product to release it under the Amicus name, and they sold the film to\n       \n        20th Century Fox\n       \n       .\n      '

this is just to try stuff.

Now we clean The files, so we first define a preprocess function to do it!

In [6]:
import io 
import nltk
import csv
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
from nltk.stem import PorterStemmer 

 
ps = PorterStemmer() 
#the fuction preprocess the string as asked in the hmk
def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [ps.stem(w) for w in tokens if not w in stopwords.words('english')]
    return " ".join(filtered_words)

We use the function on every section of the tsv files, except for the tiles of the categories, so from the part 13 and clean them.

In [25]:
for i in range(30000):
    file1 = open('Tsvfiles/film'+str(i)+'.tsv', encoding="utf8") 
    line = file1.read()# Use this to read file content as a stream: 
    words = line.split('\t') 
    for j in range(13, len(words)):
        words[j] = preprocess(words[j])
        if j ==13:
            #Here for the format of tsv files and my split('\t') the word budget would always be in my title, so I take her out
            words[j] = words[j].replace('budget ','')  
            words[j] = words[j].replace('budget\n\n','')
        print(words[j])
    tsvname = "Cleantsv/filmclean-"+str(i)+'.tsv'
    with io.open(tsvname, "w", encoding="utf-8") as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(['title','intro', 'plot', 'film_name', 'director', 'producer','writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget'])
            tsv_writer.writerow(words[13:])

10 midnight
10 midnight 1983 american crime horror thriller film 3 direct j lee thompson screenplay origin written william robert film star charl bronson lead role support cast includ lisa eilbach andrew steven gene davi geoffrey lewi wilford brimley 10 midnight releas citi film subsidiari cannon film american cinema march 11 1983
warren staci gene davi young offic equip repairman kill women reject sexual advanc attempt flirt alway seen creepi women result frequent reject 4 first victim betti june gilbert offic worker acquaint track wood area observ sex boyfriend ambush coupl kill boyfriend give chase nake woman catch stab death 4 two lo angel polic detect leo kessler charl bronson paul mcann andrew steven investig murder kessler season veteran forc mcann consider younger 4 staci avoid prosecut construct sound alibi assault victim nake except pair latex glove hide fingerprint thu minim evid lauri kessler lisa eilbach daughter leo acquaint victim student nurs becom target killer 4 mcann

In [22]:
words[13]

'10 midnight'

# EX[2]

# 2.1.1) Create your index!

Now from the clean intro and plots we start building the dictionary that will have "index":word, so we have a unique associatioin between a word in the dataset we care about and a number. Obviously we will save it as json file, as many other dictionaries that follow to be able to use them when we want in our search engine.

In [13]:
dictionar = {}
k = 0
for i in range(30000):
    file1 = open("Cleantsv/filmclean-"+str(i)+'.tsv', encoding="utf8") 
    line = file1.read()# Use this to read file content as a stream: 
    #print(line)
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    #print(wordssplitted1, wordssplitted2)
    for i in wordssplitted1:
        #print(type(i))
        if i not in dictionar:
            dictionar[i] = str(k)
            k = k+1
    for i in wordssplitted2:
        #print(type(i))
        if i not in dictionar:
            dictionar[i] = str(k)
            k = k+1
#dictionar
import json

with open('Dictionary.json', 'w') as fp:
    json.dump(dictionar, fp)

In [6]:
import pandas as pd
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary.json", 'r') as file:
    data = file.read()
diction = json.loads(data)
diction['1913']

'12741'

In [15]:
len(diction)
type(diction['hom'])

str

## Here is the inverted Dictionary

Now we build the inverted dictionary that for each index has as value the nanme of the documents we know have the word that in the previous dictionary has that unique index. We save it as json file.

In [16]:
dictionar2 = {}
length = 0
for i in range(30000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    file1 = open(file, encoding="utf8") 
    line = file1.read()# Use this to read file content as a stream: 
    #print(line)
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    #print(wordssplitted1, wordssplitted2)
    for j in wordssplitted1:
        code = diction[j]
        if code not in dictionar2:
            dictionar2[code] = [file]
        elif file not in dictionar2[code]:
            dictionar2[code].append(file)
    for j in wordssplitted2:
        code = diction[j]
        if code not in dictionar2:
            dictionar2[code] = [file]
        elif file not in dictionar2[code]:
            dictionar2[code].append(file)
#dictionar
import json

with open('Dictionary1.json', 'w') as fp:
    json.dump(dictionar2, fp)

In [5]:
import pandas as pd
import json
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary1.json", 'r') as file:
    data = file.read()
diction2 = json.loads(data)
#dictions = pd.DataFrame(diction)
#diction2['0']


In [120]:
diction['2019']
len(diction2)

114796

# 2.1.2) Execute the query

## First engine searchengine1

First we remind some funcion we need to use and the vocabularies, so the following cell must not be run, but just used to remind the instruments we are using and defining.

In [4]:
import json
with open(r"C:\Users\leona\Desktop\ADMHMK-3\dicturls.json", 'r') as file:
    data = file.read()
dicturls = json.loads(data) 
import pandas as pd
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary.json", 'r') as file:
    data = file.read()
diction = json.loads(data)
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary1.json", 'r') as file:
    data = file.read()
diction2 = json.loads(data)
import io 
import nltk
import csv
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
from nltk.stem import PorterStemmer 


 
ps = PorterStemmer() 
#the fuction preprocess the string as asked in the hmk
def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [ps.stem(w) for w in tokens if not w in stopwords.words('english')]
    return " ".join(filtered_words)

Here is the code for the engine as a function that takes a input to analyze. It will give all the documents that will match all the words of the input inside their intro or plot.

In [89]:
#HERE i DEFINE THE FUCTION TO RECEIVE THE DATASET, GIVEN AN INPUT
y = list(input().split())
def searchengine1(y):
    for i in range(len(y)):
        y[i]= preprocess(str(y[i]))
    #Now I tranform the list of input in a list of the codes in the dictiionary based on the input
    yfinal=[] #use this because some words have no match in the vocabulary
    for i in range(len(y)):
        #print(y[i])
        if y[i] in diction:
            yfinal.append(diction[y[i]])
    #Now I have to search inside the lists of values from the keys i foundb and see if some films match in the various keys.
    if  len(yfinal)<len(y):
        return print('We are sorry there are no films, in my database, that match ALL the words you gave me !(')
    else:
        starting_values = diction2[yfinal[0]]
        final_values = starting_values.copy()
        for codes in range(1,len(yfinal)):
            new = []
            #print(final_values)
            for film in final_values:                
                if film in diction2[yfinal[codes]]:
                    new.append(film)
            final_values = new
            #print(final_values)
        megaDataframe = pd.DataFrame(columns = ['Title', 'Intro', 'Url'])
        if not final_values:
            return print("Wow no film matched my quiery, I need more films to compare!")
        else:           
            k=0
            for document in final_values:
                totakeurl = document.replace('Cleantsv/filmclean-','')
                totakeurl = str(int(totakeurl.replace('.tsv', '')))
                url = dicturls[totakeurl]
                temporary = pd.read_csv('Tsvfiles/'+'film'+(totakeurl)+'.tsv',delimiter='\t' )
                title = temporary['title'][0]
                intro =  temporary['intro'][0].replace('\r\n','')
                new_row = [title, intro, url]
                megaDataframe.loc[k]=new_row
                k=k+1
            return megaDataframe
A = searchengine1(y)
A


gothic film novel


,Title,Intro,Url
0,Cross Creek (film),Cross Creek is a 1983 f...,https://en.wikipedia.org/wiki/Cross_Creek_(film)
1,Interview with the Vampire (film),Interview with the Vampire ...,https://en.wikipedia.org/wiki/Interview_with_t...
2,The Hunchback of Notre Dame (1996 film),The Hunchback of Notre Dame ...,https://en.wikipedia.org/wiki/The_Hunchback_of...
3,Big Fish,Big Fish is a 2003 Amer...,https://en.wikipedia.org/wiki/Big_Fish
4,Lemony Snicket's A Series of Unfortunate Events,Lemony Snicket's A Series of Unfortun...,https://en.wikipedia.org/wiki/Lemony_Snicket%2...
5,The Crow: Wicked Prayer,The Crow: Wicked Prayer ...,https://en.wikipedia.org/wiki/The_Crow:_Wicked...
6,Miracle at St. Anna,Miracle at St. Anna is ...,https://en.wikipedia.org/wiki/Miracle_at_St._Anna
7,Beautiful Creatures (2013 film),Beautiful Creatures is ...,https://en.wikipedia.org/wiki/Beautiful_Creatu...
8,The Scribbler (film),The Scribbler is a 2014...,https://en.wikipedia.org/wiki/The_Scribbler_(f...
9,Crimson Peak,Crimson Peak is a 2015 ...,https://en.wikipedia.org/wiki/Crimson_Peak


In [108]:
y = list(input().split())
searchengine1(y)

daniel craig


,Title,Intro,Url
0,Turner & Hooch,Turner & Hooch is a 198...,https://en.wikipedia.org/wiki/Turner_%26_Hooch
1,The Power of One (film),The Power of One is a 1...,https://en.wikipedia.org/wiki/The_Power_of_One...
2,101 Dalmatians (1996 film),101 Dalmatians is a 199...,https://en.wikipedia.org/wiki/101_Dalmatians_(...
3,Twisted Desire,Twisted Desire is a 199...,https://en.wikipedia.org/wiki/Twisted_Desire
4,The Thirteenth Floor,The Thirteenth Floor is...,https://en.wikipedia.org/wiki/The_Thirteenth_F...
5,Lara Croft: Tomb Raider,Lara Croft: Tomb Raider ...,https://en.wikipedia.org/wiki/Lara_Croft:_Tomb...
6,Road to Perdition,Road to Perdition is a ...,https://en.wikipedia.org/wiki/Road_to_Perdition
7,Casino Royale (2006 film),Casino Royale is a 2006...,https://en.wikipedia.org/wiki/Casino_Royale_(2...
8,Infamous (film),Infamous is a 2006 Amer...,https://en.wikipedia.org/wiki/Infamous_(film)
9,The Golden Compass (film),The Golden Compass is a...,https://en.wikipedia.org/wiki/The_Golden_Compa...


In [84]:
y = list(input().split())
searchengine1(y)

gothic film novel


NameError: name 'searchengine1' is not defined

# 2.2) Conjunctive query & Ranking score

## Building of the second search engine

Here we create a list with inside strings that represent intro and plot of every document.

In [15]:
Docum_and_words = []
for i in range(30000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    file1 = open(file, encoding="utf8") 
    line = file1.read()# Use this to read file content as a stream: 
    #print(line)
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    words = wordssplitted1+wordssplitted2
    A = " ".join(words)
    Docum_and_words.append(A)
#print(Docum_and_words) 
#I have the lis of documents with intro and plot



We define two function to get the tf and the number of documents containing a word.

In [3]:
import math
from textblob import TextBlob as tb
def tf(word, doc):
    return doc.count(word) / len(doc)

def n_containing(word, doclist):
    return sum(1 for doc in doclist if word in doc.split())


# Create dictionary with idf to save computational costs

# Third dictionary

Some preliminary declarations to run the code at any moment.

In [1]:
import pandas as pd
import json
Docum_and_words = []
for i in range(30000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    file1 = open(file, encoding="utf8") 
    line = file1.read()
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    words = wordssplitted1+wordssplitted2
    A = " ".join(words)
    Docum_and_words.append(A)

In [82]:
Docum_and_words[0]

'10 midnight 1983 american crime horror thriller film 3 direct j lee thompson screenplay origin written william robert film star charl bronson lead role support cast includ lisa eilbach andrew steven gene davi geoffrey lewi wilford brimley 10 midnight releas citi film subsidiari cannon film american cinema march 11 1983 warren staci gene davi young offic equip repairman kill women reject sexual advanc attempt flirt alway seen creepi women result frequent reject 4 first victim betti june gilbert offic worker acquaint track wood area observ sex boyfriend ambush coupl kill boyfriend give chase nake woman catch stab death 4 two lo angel polic detect leo kessler charl bronson paul mcann andrew steven investig murder kessler season veteran forc mcann consider younger 4 staci avoid prosecut construct sound alibi assault victim nake except pair latex glove hide fingerprint thu minim evid lauri kessler lisa eilbach daughter leo acquaint victim student nurs becom target killer 4 mcann refus go a

In [2]:
with open(r"C:\Users\leona\Desktop\ADMHMK-3\dicturls.json", 'r') as file:
    data = file.read()
dicturls = json.loads(data) 
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary.json", 'r') as file:
    data = file.read()
diction = json.loads(data) #first dict with every word and a unique value
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary1.json", 'r') as file:
    data = file.read()
diction2 = json.loads(data)#first inverted dict with number and list of document with a word that has that number in diction
import math
from textblob import TextBlob as tb
def tf(word, doc):
    return doc.count(word) / len(doc)
def n_containing(word, doclist):
    return sum(1 for doc in doclist if word in doc.split())

def idf(word, doclist):
    return math.log(len(doclist) / float(n_containing(word, doclist)))

def tfidf(word, doc, doclist):
    return (tf(word, doc) * idf(word, doclist))

This is a dictionary with inside associated to every word in diction(the one that matched indexes and words) the number of documents it is in on the 30.000 of the database.

In [34]:
ncontain = {}
for i in diction:
    refer = diction[i]
    #print(i)
    ncontain[i] = len(diction2[refer])
with open('ncontain.json', 'w') as fp:
    json.dump(ncontain, fp)
with open(r"C:\Users\leona\Desktop\ADMHMK-3\ncontain.json", 'r') as file:
    data = file.read()
ncontain = json.loads(data) #first dict with every word and a unique value

In [38]:
len(ncontain.keys())

114796

idfdict is the dictionary that if you select a word will give you its idf. We save it as a json file.

In [44]:
import math
idfdict = {}
for i in ncontain:
    idfdict[i]=math.log(30000 / ncontain[i])
with open('idfdict.json', 'w') as fp:
    json.dump(idfdict, fp)
with open(r"C:\Users\leona\Desktop\ADMHMK-3\idfdict.json", 'r') as file:
    data = file.read()
idfdict = json.loads(data)

In [61]:
with open(r"C:\Users\leona\Desktop\ADMHMK-3\idfdict.json", 'r') as file:
    data = file.read()
idfdict = json.loads(data)
#idfdict.keys()

# 2.2.1) Inverted index

## Second dictionary with the tf-idf values

We are ready to do very easily the dictionary with key : ([document, tf-idf]) because we already saved the idf in a dictioonary so the next code will be pretty fast in creating the dict. We save it a json file.

In [40]:
dictionar3 = {}
#length = 0
for i in range(30000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    temp = []
    for j in Docum_and_words[i].split():
            if j not in temp:
                #dicus = {}
                code = diction[j]
                value = tf(j, Docum_and_words[i].split())*idfdict[j]
                li = [file, value]
                if code not in dictionar3:
                    #dicus[file]=value
                    dictionar3[code] = [li]
                else:   
                    #dicus[file]=value
                    dictionar3[code].append(li)
                temp.append(j)
import json

with open('Dictionary2.json', 'w') as fp:
    json.dump(dictionar3, fp)

In [33]:
len(dictionar3)

114796

# Now I import the files I need and prepare the code for the search engine2

In [52]:
import pandas as pd
import json
with open(r"C:\Users\leona\Desktop\ADMHMK-3\dicturls.json", 'r') as file:
    data = file.read()
dicturls = json.loads(data) 
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary.json", 'r') as file:
    data = file.read()
diction = json.loads(data) #first dict with every word and a unique value
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary1.json", 'r') as file:
    data = file.read()
diction2 = json.loads(data)#first inverted dict with number and list of document with a word that has that number in diction
with open(r"C:\Users\leona\Desktop\ADMHMK-3\ncontain.json", 'r') as file:
    data = file.read()
ncontain = json.loads(data) #dict with number of times a word appear in all the cod
with open(r"C:\Users\leona\Desktop\ADMHMK-3\idfdict.json", 'r') as file:
    data = file.read()
idfdict = json.loads(data) #dict with the idf for every word
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary2.json", 'r') as file:
    data = file.read()
diction3 = json.loads(data) #second inverted dictionary wit doc and tfidf for eevry word
import math

ASbout copsine similarity I have to take the tfidf of my k best document and multipèly singularly it with the values of the quiery that are tf(relative of the quiery)*idf(relative to my documents for each component)


The query entered by the user can also be represented as a vector. We will calculate the TF*IDF for the query

TF	IDF	TF*IDF
life	0.5	1.405507153	0.702753576
learning	0.5	1.405507153	0.702753576
Let us now calculate the cosine similarity of the query and Document1. You can do the calculation using this tool.

Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

Dot product(Query, Document1) 
     = ((0.702753576) * (0.140550715) + (0.702753576)*(0.140550715))
     = 0.197545035151

||Query|| = sqrt((0.702753576)2 + (0.702753576)2) = 0.993843638185

||Document1|| = sqrt((0.140550715)2 + (0.140550715)2) = 0.198768727354

Cosine Similarity(Query, Document) = 0.197545035151 / (0.993843638185) * (0.198768727354)
                                        = 0.197545035151 / 0.197545035151
                                        = 1
        
https://janav.wordpress.com/2013/10/27/tf-idf-and-cosine-similarity/

I prepare the code for the search engine

Here there are the functions to easily compute the cosine similarity given two vectors.

In [53]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
from scipy.spatial.distance import cosine

def simple_dot(a, b):
    dsum = 0.
    for ((idx,), val) in np.ndenumerate(a):
        dsum += float(val) * float(b[idx])
    return dsum

def l2_norm(a):
    return math.sqrt(np.dot(a, a))

def cosine_similarity(a, b):
    return np.dot(a,b) / (l2_norm(a)* l2_norm(b))
np.dot([1,2],[3,4])

11

I remind that Docume_and_words is a list with inside every intro-plot(preprocessed) for every document, so I can just accees to it for my research.

In [3]:
import pandas as pd
import json
Docum_and_words = []
for i in range(30000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    file1 = open(file, encoding="utf8") 
    line = file1.read()
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    words = wordssplitted1+wordssplitted2
    A = " ".join(words)
    Docum_and_words.append(A)

In [54]:
import io 
import nltk
import csv
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
from nltk.stem import PorterStemmer 

 
ps = PorterStemmer() 
#the fuction preprocess the string as asked in the hmk
def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [ps.stem(w) for w in tokens if not w in stopwords.words('english')]
    return " ".join(filtered_words)

# 2.2.2) Execute the query

## ENGINE2!!!

Now I build the search engine2. The heap sorting is in the 'cossim' list that will have all the possible values of cosine similarity we have found for the query. We will print the dataframe with the best 20 results.

In [93]:
#HERE i DEFINE THE FUCTION TO RECEIVE THE DATASET, GIVEN AN INPUT
import heapq as hq
def tf(word, doc):
    return doc.count(word) / len(doc)
y = list(input().split())
def searchengine2(y):
    for i in range(len(y)):
        y[i]= preprocess(str(y[i]))
    #Now I tranform the list of input in a list of the codes in the dictiionary based on the input
    yfinal=[] #use this because some words have no match in the vocabulary
    for i in range(len(y)):
        #print(y[i])
        if y[i] in diction:
            yfinal.append(diction[y[i]])
    #Now I have to search inside the lists of values from the keys i foundb and see if some films match in the various keys.
    if  len(yfinal)<len(y):
        return print('We are sorry there are no films, in my database, that match ALL the words you gave me !(')
    else:
        starting_values = diction2[yfinal[0]]
        final_values = starting_values.copy()
        for codes in range(1,len(yfinal)):
            new = []
            for film in final_values:
                if film in diction2[yfinal[codes]]:
                    new.append(film)
            final_values = new
        megaDataframe = pd.DataFrame(columns = ['Title', 'Intro', 'Url', 'Similarity'])
        if not final_values:
            return print("Wow no film matched my quiery, I need more films to compare!")
        else:  
            lstofl = []
            #here there is a lstofl that has vectors associated with every document, in order of final_values
            for film in final_values:
                item = []
                for code in yfinal:
                    for value in diction3[code]:
                        if film in value:
                            item.append(value[1])
                            break
                lstofl.append(item) 
            #print(lstofl)
            #Now I have to create the inquiry vector and get the cosine similarity of beetween it and every component of lstofl 
            query = []
            for i in y:
                query.append(tf(i,y)*idfdict[i])
            cossim = []
            for vector in lstofl:
                cossim.append(cosine_similarity(query, vector))
            #print(cossim) #the cosine similariotyb in order of apparition of my document
            dict_sim = {}
            for indx in range(len(cossim)):
                sim = cossim[indx]
                if sim not in dict_sim:
                    dict_sim[sim]=[final_values[indx]]
                else:
                    dict_sim[sim].append(final_values[indx])
            #print(dict_sim)
            Peak = 20
            #HERE THE HEAP ALGORITHM
            cossim = list(set(cossim))
            to_select = hq.nlargest(Peak, cossim)   
            k=0
            #print(to_select)
            #Now i have the name key(cossim) and values(docum) and I have to take the first 15 of them.
            for i in to_select:
                if(k<Peak):
                    for document in dict_sim[i]:
                            if(k>Peak):
                                return megaDataframe
                            totakeurl = document.replace('Cleantsv/filmclean-','')
                            totakeurl = str(int(totakeurl.replace('.tsv', '')))
                            url = dicturls[totakeurl]
                            Similarity = format(i, '.10g')
                            temporary = pd.read_csv('Tsvfiles/'+'film'+(totakeurl)+'.tsv',delimiter='\t' )
                            title = temporary['title'][0]
                            intro =  temporary['intro'][0].replace('\r\n','')
                            new_row = [title, intro, url, Similarity]
                            megaDataframe.loc[k]=new_row
                            k=k+1
            return  megaDataframe
A = searchengine2(y)
A


gothic film novel


,Title,Intro,Url,Similarity
0,Dark Waters (1944 film),Dark Waters is a 1944 ...,https://en.wikipedia.org/wiki/Dark_Waters_(194...,1
1,My Name Is Julia Ross,My Name Is Julia Ross i...,https://en.wikipedia.org/wiki/My_Name_Is_Julia...,0.9999897486
2,The Beguiled (1971 film),The Beguiled is a 1971 ...,https://en.wikipedia.org/wiki/The_Beguiled_(19...,0.9999897486
3,The Scribbler (film),The Scribbler is a 2014...,https://en.wikipedia.org/wiki/The_Scribbler_(f...,0.9999897486
4,Moonfleet (1955 film),Moonfleet is a 1955 ...,https://en.wikipedia.org/wiki/Moonfleet_(1955_...,0.999958998
5,The Crow: Wicked Prayer,The Crow: Wicked Prayer ...,https://en.wikipedia.org/wiki/The_Crow:_Wicked...,0.999958998
6,Beautiful Creatures (2013 film),Beautiful Creatures is ...,https://en.wikipedia.org/wiki/Beautiful_Creatu...,0.999958998
7,Jane Eyre (2011 film),Jane Eyre is a 2011 Bri...,https://en.wikipedia.org/wiki/Jane_Eyre_(2011_...,0.9999077563
8,The Three Weird Sisters,The Three Weird Sisters ...,https://en.wikipedia.org/wiki/The_Three_Weird_...,0.9999077563
9,So Evil My Love,So Evil My Love is a 19...,https://en.wikipedia.org/wiki/So_Evil_My_Love,0.9999077563


Let's see some examples! As we use simple strings and with not many repetitions it's possible more than one value will be one, as we will put a more difficult query we will have lower similarity probably.

In [73]:
y = list(input().split())
searchengine2(y)

craig 007


,Title,Intro,Url,Similarity
0,Casino Royale (2006 film),Casino Royale is a 2006...,https://en.wikipedia.org/wiki/Casino_Royale_(2...,0.8660544238


In [107]:
y = list(input().split())
searchengine2(y)

daniel craig


,Title,Intro,Url,Similarity
0,Turner & Hooch,Turner & Hooch is a 198...,https://en.wikipedia.org/wiki/Turner_%26_Hooch,1
1,Road to Perdition,Road to Perdition is a ...,https://en.wikipedia.org/wiki/Road_to_Perdition,1
2,Defiance (2008 film),Defiance is a 2008 Amer...,https://en.wikipedia.org/wiki/Defiance_(2008_f...,1
3,I Take This Oath,I Take This Oath is a 1...,https://en.wikipedia.org/wiki/I_Take_This_Oath,1
4,Dangerous Passage,Dangerous Passage is a ...,https://en.wikipedia.org/wiki/Dangerous_Passage,1
5,Sylvia (2003 film),Sylvia is a 2003 Britis...,https://en.wikipedia.org/wiki/Sylvia_(2003_film),1
6,Layer Cake (film),Layer Cake (also occasi...,https://en.wikipedia.org/wiki/Layer_Cake_(film),1
7,The Power of One (film),The Power of One is a 1...,https://en.wikipedia.org/wiki/The_Power_of_One...,1
8,101 Dalmatians (1996 film),101 Dalmatians is a 199...,https://en.wikipedia.org/wiki/101_Dalmatians_(...,1
9,The Thirteenth Floor,The Thirteenth Floor is...,https://en.wikipedia.org/wiki/The_Thirteenth_F...,1


In [126]:
y = list(input().split())
searchengine2(y)

know light film star


,Title,Intro,Url,Similarity
0,Adventures of Captain Marvel,Adventures of Captain Marvel ...,https://en.wikipedia.org/wiki/Adventures_of_Ca...,1
1,The Hour Before the Dawn,The Hour Before the Dawn ...,https://en.wikipedia.org/wiki/The_Hour_Before_...,1
2,Journey into Light,Journey into Light is a...,https://en.wikipedia.org/wiki/Journey_into_Light,1
3,Adventures of Captain Marvel,Adventures of Captain Marvel ...,https://en.wikipedia.org/wiki/Adventures_of_Ca...,1
4,The Hour Before the Dawn,The Hour Before the Dawn ...,https://en.wikipedia.org/wiki/The_Hour_Before_...,1
5,Journey into Light,Journey into Light is a...,https://en.wikipedia.org/wiki/Journey_into_Light,1
6,Adventures of Captain Marvel,Adventures of Captain Marvel ...,https://en.wikipedia.org/wiki/Adventures_of_Ca...,1
7,The Hour Before the Dawn,The Hour Before the Dawn ...,https://en.wikipedia.org/wiki/The_Hour_Before_...,1
8,Journey into Light,Journey into Light is a...,https://en.wikipedia.org/wiki/Journey_into_Light,1
9,Surviving the Game,Surviving the Game is a...,https://en.wikipedia.org/wiki/Surviving_the_Game,1


In [128]:
y = list(input().split())
searchengine2(y)

disney movie 2019


,Title,Intro,Url,Similarity
0,Thumbelina (1994 film),Thumbelina (also known ...,https://en.wikipedia.org/wiki/Thumbelina_(1994...,1
1,The Avengers (2012 film),Marvel's The Avengers ...,https://en.wikipedia.org/wiki/The_Avengers_(20...,1
2,Cars 3,Cars 3 is a 2017 Americ...,https://en.wikipedia.org/wiki/Cars_3,0.94335701356643
3,Frozen (2013 film),Frozen is a 2013 Americ...,https://en.wikipedia.org/wiki/Frozen_(2013_film),0.89654294215813
4,The Chronicles of Narnia: Prince Caspian,The Chronicles of Narnia: Prince Casp...,https://en.wikipedia.org/wiki/The_Chronicles_o...,0.82684193281763


In [16]:
y = list(input().split())
searchengine2(y)

light light light film sun


,Title,Intro,Url,Similarity
0,Multi-Facial,Multi-Facial is a 1995 ...,https://en.wikipedia.org/wiki/Multi-Facial,0.9999935714
1,Legend (1985 film),Legend is a 1985 Americ...,https://en.wikipedia.org/wiki/Legend_(1985_film),0.999985536
2,Blade II,Blade II is a 2002 Amer...,https://en.wikipedia.org/wiki/Blade_II,0.9999598243
3,Close Encounters of the Third Kind,Close Encounters of the Third Kind ...,https://en.wikipedia.org/wiki/Close_Encounters...,0.9998698517
4,Little Monsters,Little Monsters is a 19...,https://en.wikipedia.org/wiki/Little_Monsters,0.9980234564
5,Midnight Special (film),Midnight Special is a 2...,https://en.wikipedia.org/wiki/Midnight_Special...,0.9980109468
6,Pitch Black (film),Pitch Black (titled ...,https://en.wikipedia.org/wiki/Pitch_Black_(film),0.9980109468
7,The Hallow,The Hallow (originally ...,https://en.wikipedia.org/wiki/The_Hallow,0.9948607933
8,League of Gods,League of Gods ( ...,https://en.wikipedia.org/wiki/League_of_Gods,0.9928853908
9,The Wraith,The Wraith is a 1986 ...,https://en.wikipedia.org/wiki/The_Wraith,0.9928681177


# Ex[3] build a new engine with a personal Score!

The engine will take the best 20 of engine2 and perform a score on everyone of therm, about: 
<br>
    1) the string we gave has one or more words in the title (1 word=1 point)
    <br>
    2) the string we gave has 2 or more words in the starring section (2 words = 1 point)
    <br>
    3) the film has 4 or more 'na' > -1 point
    4) the length of intro+plot is above the average of len(intro+plot) > +1 point
    <br>
    5) the length of intro+plot is below the average of len(intro+plot) > -1 point
    <br>
    6) in the string the words given match(beetween intro-plot-starring-director) more than 3 times > +1 point every word that matches at least 3 times.
    <br>
    7) in the string every word matches (beetween intro-plot-starring-director) more than 2 times > +1 point
    <br>
    

In [36]:
import pandas as pd
import json
titles = {}
for i in range(50):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    datatemp = pd.read_csv(file, delimiter = '\t')
    title =  datatemp['title'][0]
    titles[file]=title

'lone ladi'

In [51]:
import pandas as pd
import json
titles = {}
for i in range(30):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    datatemp = pd.read_csv(file, delimiter = '\t')
#datatemp['intro'][0]+datatemp['plot'][0]
for i in datatemp.values[0]:
    print(i)

eddi cruiser
eddi cruiser 1983 american film direct martin davidson screenplay written director arlen davidson base novel p f kluge sequel eddi cruiser ii eddi live follow 1989
televis report name maggi foley investig mysteri disappear legendari rock star eddi wilson flashback dramat eddi life rise fall rock roll band eddi cruiser band get start somer point new jersey club call toni mart adept write lyric eddi hire frank ridgeway aka wordman band keyboard player lyricist protest band manag doc robbin bassist sal amato round cruiser saxophonist wendel newton background singer eddi girlfriend joann carlino drummer kenni hopkin band first album tender year becam major hit record next album season hell turn nightmar inspir bleak fatalist poetri arthur rimbaud eddi push bandmat beyond limit music person eddi want great bassist sal repli great guy jersey eddi make clear band cannot great reason ever play music season hell ultim reject satin record ground dark strang earli morn hour eddi car 

In [39]:
import pandas as pd
import json
Docum_and_words = []
for i in range(30000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    file1 = open(file, encoding="utf8") 
    line = file1.read()
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    words = wordssplitted1+wordssplitted2
    A = " ".join(words)
    Docum_and_words.append(A)

In [44]:
#len(Docum_and_words[0].split())
total = 0
for i in Docum_and_words:
    total = total + len(i.split())
average = float(total)/30000

In [45]:
average

266.1692

In [112]:
import pandas as pd
import json
def score_function(doc, lizt, z, x):
    average = 266
    score = 0
    half = 0
    da = pd.read_csv(doc, delimiter = '\t')
    for i in range(len(lizt)):
        if lizt[i] in da['title'][0].split() and x == 'no':
            score = score + 1
        if lizt[i] in da['starring'][0].split():
            half = half + 1
    if z == da['language'][0]:
        score = score + 4
    if x == 'yes':
        score = score + half*7
    intro_plot = da['plot'][0]+da['intro'][0]
    value = len(intro_plot.split())
    if value > average:
        score = score + 4
    if value < (average-50):
        score = score -6
    na = 0
    for i in da.values[0]:
        if i == 'na':
            na = na +1
    if na > 3:
        score = score - 4
    return score

# Engine3!

In [114]:
#HERE i DEFINE THE FUNCTION TO RECEIVE THE DATASET, GIVEN AN INPUT
import heapq as hq
def tf(word, doc):
    return doc.count(word) / len(doc)
y = list(input('Welcome in search engine3, write a string and, after some more questions I will give you a very likely film you can be looking for!').split())
z = input('Write your preferred language')
x = input('You gave an actor/actress name in the string? type yes or no')
def searchengine3(y, z, x):
    for i in range(len(y)):
        y[i]= preprocess(str(y[i]))
    #Now I tranform the list of input in a list of the codes in the dictiionary based on the input
    yfinal=[] #use this because some words have no match in the vocabulary
    for i in range(len(y)):
        #print(y[i])
        if y[i] in diction:
            yfinal.append(diction[y[i]])
    #Now I have to search inside the lists of values from the keys i foundb and see if some films match in the various keys.
    if  len(yfinal)<len(y):
        return print('We are sorry there are no films, in my database, that match ALL the words you gave me !(')
    else:
        starting_values = diction2[yfinal[0]]
        final_values = starting_values.copy()
        for codes in range(1,len(yfinal)):
            new = []
            for film in final_values:
                if film in diction2[yfinal[codes]]:
                    new.append(film)
            final_values = new
        megaDataframe = pd.DataFrame(columns = ['Title', 'Intro', 'Url', 'Score'])
        if not final_values:
            return print("Wow no film matched my quiery, I need more films to compare!")
        else:  
            dict_score = {}
            score_list = []
            Threshold  = 2
            k_limit = 19
            #print(final_values)
            for doc in final_values:
                #score_function is the function I create and used to give points to every film
                x = preprocess(x)
                z = preprocess(z)
                score = score_function(doc, y, z, x)
                if score not in dict_score:
                    dict_score[score]=[doc]
                    score_list.append(score)
                else:
                    dict_score[score].append(doc)
            score_list = list(set(score_list))
            best = hq.nlargest(Threshold, score_list)
            k = 0
            for score in best:
                if k < k_limit :
                    for document in dict_score[score]:
                        if k < k_limit:
                            totakeurl = document.replace('Cleantsv/filmclean-','')
                            totakeurl = str(int(totakeurl.replace('.tsv', '')))
                            url = dicturls[totakeurl]
                            Score = score
                            temporary = pd.read_csv('Tsvfiles/'+'film'+(totakeurl)+'.tsv',delimiter='\t' )
                            title = temporary['title'][0]
                            intro =  temporary['intro'][0].replace('\r\n','')
                            new_row = [title, intro, url, Score]
                            megaDataframe.loc[k]=new_row 
                            k = k+1
                        else:
                            return megaDataframe
                else:
                    return megaDataframe
                
            
A = searchengine3(y, z, x)
A

Welcome in search engine3, write a string and, after some more questions I will give you a very likely film you can be looking for!daniel craig 
Write your preferred languageenglish
You gave an actor/actress name in the string? type yes or noyes


,Title,Intro,Url,Score
0,Turner & Hooch,Turner & Hooch is a 198...,https://en.wikipedia.org/wiki/Turner_%26_Hooch,8
1,The Power of One (film),The Power of One is a 1...,https://en.wikipedia.org/wiki/The_Power_of_One...,8
2,101 Dalmatians (1996 film),101 Dalmatians is a 199...,https://en.wikipedia.org/wiki/101_Dalmatians_(...,8
3,Twisted Desire,Twisted Desire is a 199...,https://en.wikipedia.org/wiki/Twisted_Desire,8
4,The Thirteenth Floor,The Thirteenth Floor is...,https://en.wikipedia.org/wiki/The_Thirteenth_F...,8
5,Lara Croft: Tomb Raider,Lara Croft: Tomb Raider ...,https://en.wikipedia.org/wiki/Lara_Croft:_Tomb...,8
6,Road to Perdition,Road to Perdition is a ...,https://en.wikipedia.org/wiki/Road_to_Perdition,8
7,Casino Royale (2006 film),Casino Royale is a 2006...,https://en.wikipedia.org/wiki/Casino_Royale_(2...,8
8,Infamous (film),Infamous is a 2006 Amer...,https://en.wikipedia.org/wiki/Infamous_(film),8
9,The Golden Compass (film),The Golden Compass is a...,https://en.wikipedia.org/wiki/The_Golden_Compa...,8


In [98]:
y = list(input('Welcome in search engine3, write a string and, after some more questions I will give you a very likely film you can be looking for!').split())
z = input('Write your preferred language')
x = input('You gave an actor/actress name in the string? type yes or no')
searchengine3(y, z, x)

Welcome in search engine3, write a string and, after some more questions I will give you a very likely film you can be looking for!james bond
Write your preferred languageenglish
You gave an actor/actress name in the string? type yes or nono


,Title,Intro,Url,Score
0,Coming to America,Coming to America is a ...,https://en.wikipedia.org/wiki/Coming_to_America,8
1,Soul Food (film),Soul Food is a 1997 Ame...,https://en.wikipedia.org/wiki/Soul_Food_(film),8
2,Ninja Assassin,Ninja Assassin is a 200...,https://en.wikipedia.org/wiki/Ninja_Assassin,8
3,Kung Fu Panda 3,Kung Fu Panda 3 is a 20...,https://en.wikipedia.org/wiki/Kung_Fu_Panda_3,8
4,Deepwater Horizon (film),Deepwater Horizon is a ...,https://en.wikipedia.org/wiki/Deepwater_Horizo...,8


In [100]:
y = list(input('Welcome in search engine3, write a string and, after some more questions I will give you a very likely film you can be looking for!').split())
z = input('Write your preferred language')
x = input('You gave an actor/actress name in the string? type yes or no')
searchengine3(y, z, x)

Welcome in search engine3, write a string and, after some more questions I will give you a very likely film you can be looking for!casino royale
Write your preferred languageenaglish
You gave an actor/actress name in the string? type yes or nono


,Title,Intro,Url,Score
0,Casino Royale (2006 film),Casino Royale is a 2006...,https://en.wikipedia.org/wiki/Casino_Royale_(2...,6
1,Casino Royale (1967 film),Casino Royale is a 1967...,https://en.wikipedia.org/wiki/Casino_Royale_(1...,6
2,Quantum of Solace,Quantum of Solace is a ...,https://en.wikipedia.org/wiki/Quantum_of_Solace,4
3,Thunderball (film),Thunderball is a 1965 B...,https://en.wikipedia.org/wiki/Thunderball_(film),4
4,Khartoum (film),Khartoum is a 1966 film...,https://en.wikipedia.org/wiki/Khartoum_(film),4
